<a href="https://colab.research.google.com/github/DmitriyValetov/nlp_course_project/blob/master/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install rouge_score
import torch
from rouge_score import rouge_scorer
from pprint import pprint

def metrics(bx2, by2, eos=3):
  scores = {}  # score: [value, value, ..., batch_size]
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
  ps = by2.detach().cpu().numpy()  # batch predictions
  ts = bx2.detach().cpu().numpy()  # batch targets
  for p, t in zip(ps, ts):
    # target to set
    t_eos_i = np.where(t==eos)[0][0] if eos in t else len(t)
    t_eos = t[:t_eos_i]
    t_set = set(t_eos) - {0, 1, 2, 3}
    # prediction to set
    p_eos_i = np.where(p==eos)[0][0] if eos in p else len(p)
    p_eos = p[:p_eos_i]
    p_set = set(p_eos) - {0, 1, 2, 3}
    # metrics
    i_set = t_set.intersection(p_set)
    pre = len(i_set)/len(p_set) if len(p_set) != 0 else 0
    rec = len(i_set)/len(t_set) if len(t_set) != 0 else 0
    f1 = 2*pre*rec/(pre + rec) if pre + rec != 0 else 0
    rl = len(p_eos)/len(t_eos) if len(t_eos) != 0 else 0
    # to string for fouge scorer
    t_str = ' '.join(map(str, t_eos))
    p_str = ' '.join(map(str, p_eos))
    rs = scorer.score(t_str, p_str)
    print(f't: {t}, p: {p}')
    print(f'eos in t?: {eos in st}, p_eos_i: {st_eos_i}')
    print(f'eos in p?: {eos in sp}, sp_eos_i: {sp_eos_i}')
    print(f't_eos: {t_eos}, p_eos: {p_eos}')
    print(f't_set: {t_set}, p_set: {p_set}')
    print(f't_str: "{t_str}", p_str: "{p_str}"')
    pprint(rs)
    print(f'precision: {pre}, recall: {rec}, f1: {f1}. rl: {rl}\n')
    scores.setdefault('set_precision', []).append(pre)
    scores.setdefault('set_recall', []).append(rec)
    scores.setdefault('set_f1', []).append(f1)
    scores.setdefault('relen', []).append(rl)
    for k, v in rs.items():
      scores.setdefault(f'{k}_precision', []).append(v.precision)
      scores.setdefault(f'{k}_recall', []).append(v.recall)
      scores.setdefault(f'{k}_f1', []).append(v.fmeasure)
  return scores

bx2 = torch.tensor([[1, 5, 3, 4, 4, 3, 3, 3],  # target 1
                   [5, 3, 0, 4, 3, 0, 0, 0]])  # target 2
by2 = torch.tensor([[0, 1, 4, 3, 4, 7, 4],  # predition 1
                   [0, 5, 4, 3, 3, 0, 3]])  # predition 2
metrics(bx2, by2)

t: [1 5 3 4 4 3 3 3], p: [0 1 4 3 4 7 4]
eos in t?: True, p_eos_i: 2
eos in p?: True, sp_eos_i: 2
t_eos: [1 5], p_eos: [0 1 4]
t_set: {5}, p_set: {4}
t_str: "1 5", p_str: "0 1 4"
{'rouge1': Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4)}
precision: 0.0, recall: 0.0, f1: 0. rl: 1.5

t: [5 3 0 4 3 0 0 0], p: [0 5 4 3 3 0 3]
eos in t?: True, p_eos_i: 2
eos in p?: True, sp_eos_i: 2
t_eos: [5], p_eos: [0 5 4]
t_set: {5}, p_set: {4, 5}
t_str: "5", p_str: "0 5 4"
{'rouge1': Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.3333333333333333, recall=1.0, fmeasure=0.5)}
precision: 0.5, recall: 1.0, f1: 0.6666666666666666. rl: 3.0



{'relen': [1.5, 3.0],
 'rouge1_f1': [0.4, 0.5],
 'rouge1_precision': [0.3333333333333333, 0.3333333333333333],
 'rouge1_recall': [0.5, 1.0],
 'rouge2_f1': [0.0, 0.0],
 'rouge2_precision': [0.0, 0.0],
 'rouge2_recall': [0.0, 0.0],
 'rougeL_f1': [0.4, 0.5],
 'rougeL_precision': [0.3333333333333333, 0.3333333333333333],
 'rougeL_recall': [0.5, 1.0],
 'set_f1': [0, 0.6666666666666666],
 'set_precision': [0.0, 0.5],
 'set_recall': [0.0, 1.0]}